<a href="https://colab.research.google.com/github/Tsunami-Song/DSDNE/blob/main/experiment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from __future__ import print_function, division
import numpy as np 
import pandas as pd 
import os
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(15)

Mounted at /content/drive


In [2]:
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
fea_col = ["txId","time_step"] + tx_features + agg_features
features = pd.read_csv("/content/drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_features.csv",header=None,names=fea_col)
classes = pd.read_csv("/content/drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")

## RF(AF)

In [20]:
X = data[tx_features+agg_features]
y = data['class']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0,shuffle=False)
clf = RandomForestClassifier(n_estimators=50, max_depth=100,random_state=15).fit(X_train,y_train)
preds = clf.predict(X_test)
prec,rec,f1,num = precision_recall_fscore_support(y_test,preds, average=None)
print("AF RandomForest Results")
print("Precision:%.3f \nRecall:%.3f \nF1 Score:%.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test,preds,average='micro')
print("Micro-Average F1 Score:",micro_f1)

AF RandomForest Results
Precision:0.983 
Recall:0.650 
F1 Score:0.782
Micro-Average F1 Score: 0.9772369362920544


## RF(AF+SDNE)

In [18]:
emb = pd.read_csv("/content/drive/MyDrive/result/sdne.txt")
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
embed_names = ["emb_feat_"+str(i) for i in range(1,65)]
data = pd.merge(features,emb,on='txId')
data = pd.merge(data,classes,on='txId')
data.columns = ["txId","time_step"] + tx_features + agg_features + embed_names + ['class']
data = data[(data['class']==0) | (data['class']==1)]
X = data[tx_features+agg_features+embed_names]
y = data['class']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0,shuffle=False)
clf = RandomForestClassifier(n_estimators=50, max_depth=100,random_state=15).fit(X_train,y_train)
preds = clf.predict(X_test)
prec,rec,f1,num = precision_recall_fscore_support(y_test,preds, average=None)
print("AF+SDNE RandomForest Results")
print("Precision:%.3f \nRecall:%.3f \nF1 Score:%.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test,preds,average='micro')
print("Micro-Average F1 Score:",micro_f1)

AF+SDNE RandomForest Results
Precision:0.991 
Recall:0.650 
F1 Score:0.785
Micro-Average F1 Score: 0.9775948460987831


## RF(AF+DSDNE)

In [16]:
emb = pd.read_csv("/content/drive/MyDrive/SDNE/sdne_feature_stack2_sdne.txt")
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
embed_names = ["emb_feat_"+str(i) for i in range(1,65)]
data = pd.merge(features,emb,on='txId')
data = pd.merge(data,classes,on='txId')
data.columns = ["txId","time_step"] + tx_features + agg_features + embed_names + ['class']
data = data[(data['class']==0) | (data['class']==1)]
X = data[tx_features+agg_features+embed_names]
y = data['class']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0,shuffle=False)
clf = RandomForestClassifier(n_estimators=50, max_depth=100,random_state=15).fit(X_train,y_train)
preds = clf.predict(X_test)
prec,rec,f1,num = precision_recall_fscore_support(y_test,preds, average=None)
print("AF+SDNE RandomForest Results")
print("Precision:%.3f \nRecall:%.3f \nF1 Score:%.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test,preds,average='micro')
print("Micro-Average F1 Score:",micro_f1)

AF+SDNE RandomForest Results
Precision:0.986 
Recall:0.655 
F1 Score:0.787
Micro-Average F1 Score: 0.9777380100214745
